In [1]:
from yaml import load, dump
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper
import re
import itertools

In [2]:
all_data = load(open('vectors.yml').read(),Loader=Loader)

In [3]:
def parse(val_name,schema):
    vals = schema[val_name]
    result = []
    for v in vals:
        sub_vals = re.findall('%([^;]+);',v)
        if len(sub_vals)==0:
            result.append(v)
            continue
        sub_arrays = []
        for sub_val in sub_vals:
            sub_arrays.append(parse(sub_val,schema))

        possible_values = itertools.product(*sub_arrays)
        for x in possible_values:
            generating_v = v
            for sub_val_name,sub_val_value in zip(sub_vals,x):
                generating_v = generating_v.replace('%%%s;'%sub_val_name,sub_val_value)
            result.append(generating_v)
    return result

In [4]:
all_vectors = parse('all_vectors',all_data)
all_data['oracle_select'] = []
without_oracle = parse('all_vectors',all_data)

In [5]:
def save(filename,vectors):
    res_file = open(filename,'wb')
    res_file.write(('\n'.join(vectors)).encode('ascii'))
    res_file.close()

In [8]:
save('time_vectors.txt',all_vectors)
save('time_vectors_without_oracle.txt',without_oracle)